In [ ]:
# copiar esto: !pip install qiskit
import sys
import numpy as np
import math

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, BasicAer, IBMQ
from qiskit.visualization import plot_histogram, plot_bloch_multivector

from qiskit.extensions import Initialize

In [ ]:
def encode_bitstring(bitstring, qr, cr, inverse=False):
    """
    Construye la superposición cuantica de la cadena de bits
    """
    n = math.ceil(math.log2(len(bitstring))) + 2                 #número de qbits
    assert n > 2, "la longitud de la cadena debe ser almenos 2"
    
    qc = QuantumCircuit(qr, cr)
    
    #Amplitud de probabilidad del estado deseado
    desired_vector = np.array([ 0.0 for i in range(2**n) ])     #inicializando a cero

################################################################################################################    
 #         Cambio requerido en el código para corregir error presentado   #
    
    qc_init = QuantumCircuit(n) # Creación de compuertas circuitales para inicialización
    inverse_qc_init = QuantumCircuit(n) # Creación de compuertas circuitales para inversión de la inicialización

################################################################################################################

    amplitude = np.sqrt(1.0/len(bitstring))
    

    #Se usa la misma lógica que en el procedimiento ejemplo, sin embargo se le debe añadir un cambio debido a 
    #que no tenemos solo dos tipos de valores “M” o “-”, Ahora tenemos 4 “A”, “T”, “G” y  “C”, por lo que se 
    #le hizo este cambio al codigo de codificación para agregar cada caso:
    
    for i, b in enumerate(bitstring):   #bucle para establecer casos para los valores A,C,G,T
        pos = i * 4
        if  b == "C":
            pos += 1
        elif b =="G":
            pos += 2
        elif b =="T":
            pos += 3
        desired_vector[pos] = amplitude

################################################################################################################    
 #         Cambio requerido en el código para corregir error presentado   #
    
    qc_init = Initialize(desired_vector)
    
 ################################################################################################################       
    
    if not inverse:
#         qc.initialize(desired_vector, qr)

        qc.append(qc_init, qr)  #  Cambio requerido en el código para corregir error presentado   #
    
        qc.barrier(qr)
        
    else:
#         qc.initialize(desired_vector, qr).inverse()  #invertir el circuito

        inverse_qc_init = qc_init.gates_to_uncompute()  # Cambio requerido en el código para corregir error presentado   #
        qc.append(inverse_qc_init, qr)  # Cambio requerido en el código para corregir error presentado   #
        
        qc.barrier(qr)
        for i in range(n):
            qc.measure(qr[i], cr[i])
    #print(desired_vector)
    return qc

In [ ]:
GOAT     = "ATGGTGCTGTCTGCCGCCGACAAGTCCAATGTCAAGGCCGCCTGGGGCAAGGTTGGCAGCAACGCTGGAGCTTATGGCGCAGAGGCTCTGGAGAGGATGTTCCTGAGCTTCCCCACCACCAAGACCTACTTCCCCCACTTCGACCTGAGCCACGGCTCGGCCCAGGTCAAGGGCCACGGCGAGAAGGTGGCCGCCGCGCTGACCAAAGCGGTGGGCCACCTGGACGACCTGCCCGGTACTCTGTCTGATCTGAGTGACCTGCACGCCCACAAGCTGCGTGTGGACCCGGTCAACTTTAAGCTTCTGAGCCACTCCCTGCTGGTGACCCTGGCCTGCCACCACCCCAGTGATTTCACCCCCGCGGTCCACGCCTCCCTGGACAAGTTCTTGGCCAACGTGAGCACCGTGCTGACCTCCAAATACCGTTAA"
HORSE    = "ATGGTGCTGTCTGCCGCCGACAAGACCAACGTCAAGGCCGCCTGGAGTAAGGTTGGCGGCCACGCTGGCGAGTTTGGCGCAGAGGCCCTAGAGAGGATGTTCCTGGGCTTCCCCACCACCAAGACCTACTTCCCCCACTTCGATCTGAGCCACGGCTCCGCCCAGGTCAAGGCCCACGGCAAGAAGGTGGGCGACGCGCTGACTCTCGCCGTGGGCCACCTGGACGACCTGCCTGGCGCCCTGTCGAATCTGAGCGACCTGCACGCACACAAGCTGCGCGTGGACCCCGTCAACTTCAAGCTTCTGAGTCATTGCCTGCTGTCCACCTTGGCCGTCCACCTCCCCAACGATTTCACCCCTGCCGTCCACGCCTCCCTGGACAAGTTCTTGAGCAGTGTGAGCACCGTGCTGACCTCCAAATACCGTTAA" 
PIDGEOT  = "ATGGTGCTGTCTGCCAACGACAAGAGCAACGTGAAGGCCGTCTTCGGCAAAATCGGCGGCCAGGCCGGTGACTTGGGTGGTGAAGCCCTGGAGAGGTTGTTCATCACCTACCCCCAGACCAAGACCTACTTCCCCCACTTCGACCTGTCACATGGCTCCGCTCAGATCAAGGGGCACGGCAAGAAGGTGGCGGAGGCACTGGTTGAGGCTGCCAACCACATCGATGACATCGCTGGTGCCCTCTCCAAGCTGAGCGACCTCCACGCCCAAAAGCTCCGTGTGGACCCCGTCAACTTCAAACTGCTGGGTCACTGCTTCCTGGTGGTCGTGGCCGTCCACTTCCCCTCTCTCCTGACCCCGGAGGTCCATGCTTCCCTGGACAAGTTCGTGTGTGCCGTGGGCACCGTCCTTACTGCCAAGTACCGTTAA"



In [ ]:
n = math.ceil(math.log2(len(GOAT))) + 2                 #Número de Qubits
qr = QuantumRegister(n)
cr = ClassicalRegister(n)

qc_GOAT    = encode_bitstring(GOAT, qr, cr)
qc_HORSE    = encode_bitstring(HORSE, qr, cr)
qc_PIDGEOT   = encode_bitstring(PIDGEOT, qr, cr)


#Creación de cicuitos para cada secuencia y se guardan en un diccionario.
circs = {"GOAT": qc_GOAT, "HORSE": qc_HORSE, "PIDGEOT": qc_PIDGEOT }

In [ ]:

inverse_qc_GOAT    = encode_bitstring(GOAT, qr, cr , inverse=True)
inverse_qc_HORSE    = encode_bitstring(HORSE, qr, cr, inverse=True)
inverse_qc_PIDGEOT     = encode_bitstring(PIDGEOT , qr, cr, inverse=True)

#Creación de cicuitos inversos para cada secuencia y se guardan en un diccionario.
inverse_circs = {"GOAT": inverse_qc_GOAT, "HORSE": inverse_qc_HORSE, "PIDGEOT": inverse_qc_PIDGEOT }

In [ ]:
from qiskit import IBMQ, BasicAer

key = "GOAT"       #El nombre de la cadena genética usada para encontrar otras similares.
other_key = key;
# Simulador Local
backend = BasicAer.get_backend("qasm_simulator")
shots = 8192

combined_circs = {}
count = {}

most_similar, most_similar_score = "", -1.0

        
combined_circs[other_key] = circs[key].compose(inverse_circs[other_key])   #Circuito combinado para buscar circuitos similares
job = execute(combined_circs[other_key], backend=backend,shots=shots)
st = job.result().get_counts(combined_circs[other_key])
if "0"*n in st:
    sim_score = st["0"*n]/shots
else:
    sim_score = 0.0
    
print("Similarity score of",key,"and",other_key,"is",sim_score)

Similarity score of GOAT and GOAT is 1.0
[ANSWER] GOAT is most similar to GOAT


In [ ]:
from qiskit import IBMQ, BasicAer
GOAT_MOD = "ATGGTGCTGTCTGCCGCCGACAAGTCCAATGTCAAGGCCGCCTGGGGCAAGGTTGGCAGCAACGCTGGAGCTTATGGCGCAGAGGCTCTGGAGAGGATGTTCCTGAGCTACCCCACCACCAAGACCTACTTCCCCCACTTCGACCTGAGCCACGGCTCGGCCCAGGTCAAGGGCCACGGCGAGAAGGTGGCCGCCGCGCTGACCAAAGCGGTGGGCCACCTGGACGACCTGCCCGGTACTCTGTCTGATCTGAGTGACCTGCACGCCCACAAGCTGCGTGTGGACCCGGTCAACTTTAAGCTTCTGAGCCACTCCCTGCTGGTGACCCTGGCCTGCCACCACCCCAGTGATTTCACCCCCGCGGTCCACGCCTCCCTGGACAAGTTCTTGGCCAACGTGAGCACCGTGCTGACCTCCAAATACCGTTAA"
qc_GOAT_MOD   = encode_bitstring(GOAT_MOD, qr, cr)
inverse_qc_GOAT_MOD     = encode_bitstring(GOAT_MOD , qr, cr, inverse=True)

key = "GOAT"       #El nombre de la cadena genética usada para encontrar otras similares.
other_key = "GOAT_MOD";
# Simulador Local
backend = BasicAer.get_backend("qasm_simulator")
shots = 8192

combined_circs = {}
count = {}

most_similar, most_similar_score = "", -1.0

        
combined_circs[other_key] = circs[key].compose(inverse_qc_GOAT_MOD)  #Circuito combinado para buscar circuitos similares
job = execute(combined_circs[other_key], backend=backend,shots=shots)
st = job.result().get_counts(combined_circs[other_key])
if "0"*n in st:
    sim_score = st["0"*n]/shots
else:
    sim_score = 0.0
    
print("Similarity score of",key,"and",other_key,"is",sim_score)

Similarity score of GOAT and GOAT_MOD is 0.9949951171875


In [ ]:
from qiskit import IBMQ, BasicAer

key = "GOAT"       #El nombre de la cadena genética usada para encontrar otras similares.

# Simulador Local
backend = BasicAer.get_backend("qasm_simulator")
shots = 8192

combined_circs = {}
count = {}

most_similar, most_similar_score = "", -1.0

for other_key in inverse_circs:
    if other_key == key:
        continue
        
    combined_circs[other_key] = circs[key].compose(inverse_circs[other_key])   #Circuito combinado para buscar circuitos similares
    job = execute(combined_circs[other_key], backend=backend,shots=shots)
    st = job.result().get_counts(combined_circs[other_key])
    if "0"*n in st:
        sim_score = st["0"*n]/shots
    else:
        sim_score = 0.0
    
    print("Similarity score of",key,"and",other_key,"is",sim_score)
    if most_similar_score < sim_score:
        most_similar, most_similar_score = other_key, sim_score

print("[ANSWER]", key,"is most similar to", most_similar)

key = "HORSE"        #El nombre de la cadena genética usada para encontrar otras similares.


# Simulador Local
backend = BasicAer.get_backend("qasm_simulator")
shots = 8192

combined_circs = {}
count = {}

most_similar, most_similar_score = "", -1.0

for other_key in inverse_circs:
    if other_key == key:
        continue
        
    combined_circs[other_key] = circs[key].compose(inverse_circs[other_key])   #Circuito combinado para buscar circuitos similares
    job = execute(combined_circs[other_key], backend=backend,shots=shots)
    st = job.result().get_counts(combined_circs[other_key])
    if "0"*n in st:
        sim_score = st["0"*n]/shots
    else:
        sim_score = 0.0
    
    print("Similarity score of",key,"and",other_key,"is",sim_score)
    if most_similar_score < sim_score:
        most_similar, most_similar_score = other_key, sim_score

print("[ANSWER]", key,"is most similar to", most_similar)

Similarity score of GOAT and HORSE is 0.767822265625
Similarity score of GOAT and PIDGEOT is 0.53466796875
[ANSWER] GOAT is most similar to HORSE
Similarity score of HORSE and GOAT is 0.772705078125
Similarity score of HORSE and PIDGEOT is 0.5574951171875
[ANSWER] HORSE is most similar to GOAT
